In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

PHI3 - COVID19

In [5]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import pickle

# Load PHI-3 model and tokenizer
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
phi3_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

responses = []

def format_prompt(text):
    return "You are an expert misinformation detection assistant.\n\n"\
           "Your task is read the following text and determine which one "\
           "of these categories best describes its author:\n\n"\
           "Religious Conspiracy Theorist\n"\
           "Misinformation Spreader\n"\
           "Anti Vaxxer\n"\
           "Fear Mongerer\n\n"\
           "Text:\n" + text + "\n"\
           "Respond with exactly one of these labels—and nothing else: "

@torch.no_grad()
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(phi3_model.device)

    output = phi3_model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False  # Fix for DynamicCache issue
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    reply = decoded.strip()
    responses.append(reply)

    reply_lower = reply.lower()
    if "conspiracy" in reply_lower:
        return 0
    elif "misinformation" in reply_lower:
        return 1
    elif "vaxx" in reply_lower or "vaccine" in reply_lower:
        return 2
    elif "fear" in reply_lower:
        return 3
    else:
        return -1

def main():
    base_folder = '/kaggle/input/owaiskkk/VaxGaurd-Dataset-roles (New)/VaxGaurd-Dataset-roles (New)/'
    files = [
        [
            'GPT-3.5/COVID-19/clean_COVID-19_texts_GPT-3.5_Anti-Vacciner.csv',
            'GPT-3.5/COVID-19/clean_COVID-19_texts_GPT-3.5_fear mongerer.csv',
            'GPT-3.5/COVID-19/clean_COVID-19_texts_GPT-3.5_Misinformation spreader.csv',
            'GPT-3.5/COVID-19/clean_COVID-19_texts_GPT-3.5_religious Conspiracy theorist.csv'
        ],
        [
            'GPT-4o/COVID-19/clean_COVID-19_texts_GPT-4o_Anti-Vacciner.csv',
            'GPT-4o/COVID-19/clean_COVID-19_texts_GPT-4o_fear mongerer.csv',
            'GPT-4o/COVID-19/clean_COVID-19_texts_GPT-4o_Misinformation spreader.csv',
            'GPT-4o/COVID-19/clean_COVID-19_texts_GPT-4o_religious Conspiracy theorist.csv'
        ],
        [
            'Llama 3/COVID-19/clean_COVID-19_texts_llama3_Anti-Vacciner.csv',
            'Llama 3/COVID-19/clean_COVID-19_texts_llama3_Misinformation spreader.csv',
            'Llama 3/COVID-19/clean_COVID-19_texts_llama_fear_monger.csv',
            'Llama 3/COVID-19/clean_COVID-19_texts_llama_religious.csv'
        ],
        [
            'Mistral/COVID-19/clean_COVID-19_texts_mistral_Anti-Vacciner.csv',
            'Mistral/COVID-19/clean_COVID-19_texts_mistral_Misinformation spreader.csv',
            'Mistral/COVID-19/clean_COVID-19_texts_mistral_fear.csv',
            'Mistral/COVID-19/clean_COVID-19_texts_mistral_religious.csv'
        ]
    ]

    # ✅ Corrected indentation
    files = [[base_folder + csv_file for csv_file in model] for model in files]

    models = ['GPT-3.5', 'GPT-4o', 'Llama 3', 'Mistral']
    for i, model_files in enumerate(files):
        texts = []
        labels = []
        model = models[i]

        for j, file in enumerate(model_files):
            df = pd.read_csv(file, encoding='latin1')
            col = 'Misinformation' if 'Misinformation' in df.columns else df.columns[0]
            df = df[col].dropna()
            texts.extend(df.tolist())
            labels.extend([j] * len(df))

        preds = []
        for text in tqdm(texts, desc=f'Model: {model}'):
            preds.append(classify(text))

        with open(f'responses_COVID_{model}.pickle', 'wb') as f:
            pickle.dump(responses, f)
            responses.clear()

        with open(f'COVID_{model}.pickle', 'wb') as f:
            pickle.dump(texts, f)
            pickle.dump(labels, f)
            pickle.dump(preds, f)

        overall_acc = accuracy_score(labels, preds)
        overall_prec = precision_score(labels, preds, average="macro", zero_division=0)
        overall_rec = recall_score(labels, preds, average="macro", zero_division=0)
        overall_f1 = f1_score(labels, preds, average="macro", zero_division=0)

        print(f'\n{model} Results:')
        print(f'Accuracy:  {overall_acc:.4f}')
        print(f'Precision: {overall_prec:.4f}')
        print(f'Recall:    {overall_rec:.4f}')
        print(f'F1 Score:  {overall_f1:.4f}')

if __name__ == '__main__':
    main()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...


GPT-4o Results:
Accuracy:  0.2509
Precision: 0.0627
Recall:    0.2500
F1 Score:  0.1003


Model: Llama 3: 100%|██████████| 2000/2000 [22:05<00:00,  1.51it/s]



Llama 3 Results:
Accuracy:  0.2500
Precision: 0.0625
Recall:    0.2500
F1 Score:  0.1000


Model: Mistral: 100%|██████████| 1946/1946 [17:46<00:00,  1.82it/s]


Mistral Results:
Accuracy:  0.2441
Precision: 0.0610
Recall:    0.2500
F1 Score:  0.0981


PHI3 - HPV

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import pickle

# Load PHI-3 model and tokenizer
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
phi3_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

responses = []

def format_prompt(text):
    return "You are an expert misinformation detection assistant.\n\n"\
           "Your task is read the following text and determine which one "\
           "of these categories best describes its author:\n\n"\
           "Religious Conspiracy Theorist\n"\
           "Misinformation Spreader\n"\
           "Anti Vaxxer\n"\
           "Fear Mongerer\n\n"\
           "Text:\n" + text + "\n"\
           "Respond with exactly one of these labels—and nothing else: "

@torch.no_grad()
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(phi3_model.device)

    output = phi3_model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False  # Fix for DynamicCache issue
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    reply = decoded.strip()
    responses.append(reply)

    reply_lower = reply.lower()
    if "conspiracy" in reply_lower:
        return 0
    elif "misinformation" in reply_lower:
        return 1
    elif "vaxx" in reply_lower or "vaccine" in reply_lower:
        return 2
    elif "fear" in reply_lower:
        return 3
    else:
        return -1

def main():
    base_folder = '/kaggle/input/owaiskkk/VaxGaurd-Dataset-roles (New)/VaxGaurd-Dataset-roles (New)/'
    files = [
        [
            'GPT-3.5/HPV/clean_HPV_texts_GPT-3.5_Anti-Vacciner.csv',
            'GPT-3.5/HPV/clean_HPV_texts_GPT-3.5_Misinformation spreader.csv',
            'GPT-3.5/HPV/clean_HPV_texts_GPT-3.5_fear mongerer.csv',
            'GPT-3.5/HPV/clean_HPV_texts_GPT-3.5_religious Conspiracy theorist.csv'
        ],
        [
            'GPT-4o/HPV/clean_HPV_texts_GPT-4o_Anti-Vacciner.csv',
            'GPT-4o/HPV/clean_HPV_texts_GPT-4o_Misinformation spreader.csv',
            'GPT-4o/HPV/clean_HPV_texts_GPT-4o_fear mongerer.csv',
            'GPT-4o/HPV/clean_HPV_texts_GPT-4o_religious Conspiracy theorist.csv'
        ],
        [
            'Llama 3/HPV/clean_HPV_texts_llama3_Anti-Vacciner.csv',
            'Llama 3/HPV/clean_HPV_texts_llama3_Misinformation spreader.csv',
            'Llama 3/HPV/clean_HPV_texts_llama_fear_monger.csv',
            'Llama 3/HPV/clean_HPV_texts_llama_religious.csv'
        ],
        [
            'Mistral/HPV/clean_HPV_texts_mistral_Anti-Vacciner.csv',
            'Mistral/HPV/clean_HPV_texts_mistral_Misinformation spreader.csv',
            'Mistral/HPV/clean_HPV_texts_mistral_fear.csv',
            'Mistral/HPV/clean_HPV_texts_mistral_religious.csv'
        ]
    ]

    # ✅ Corrected indentation
    files = [[base_folder + csv_file for csv_file in model] for model in files]

    models = ['GPT-3.5', 'GPT-4o', 'Llama 3', 'Mistral']
    for i, model_files in enumerate(files):
        texts = []
        labels = []
        model = models[i]

        for j, file in enumerate(model_files):
            df = pd.read_csv(file, encoding='latin1')
            col = 'Misinformation' if 'Misinformation' in df.columns else df.columns[0]
            df = df[col].dropna()
            texts.extend(df.tolist())
            labels.extend([j] * len(df))

        preds = []
        for text in tqdm(texts, desc=f'Model: {model}'):
            preds.append(classify(text))

        with open(f'responses_COVID_{model}.pickle', 'wb') as f:
            pickle.dump(responses, f)
            responses.clear()

        with open(f'COVID_{model}.pickle', 'wb') as f:
            pickle.dump(texts, f)
            pickle.dump(labels, f)
            pickle.dump(preds, f)

        overall_acc = accuracy_score(labels, preds)
        overall_prec = precision_score(labels, preds, average="macro", zero_division=0)
        overall_rec = recall_score(labels, preds, average="macro", zero_division=0)
        overall_f1 = f1_score(labels, preds, average="macro", zero_division=0)

        print(f'\n{model} Results:')
        print(f'Accuracy:  {overall_acc:.4f}')
        print(f'Precision: {overall_prec:.4f}')
        print(f'Recall:    {overall_rec:.4f}')
        print(f'F1 Score:  {overall_f1:.4f}')

if __name__ == '__main__':
    main()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model: GPT-3.5:  40%|███▉      | 792/2000 [07:10<11:00,  1.83it/s]

PHI3 - influenza 

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import pickle

# Load PHI-3 model and tokenizer
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
phi3_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

responses = []

def format_prompt(text):
    return "You are an expert misinformation detection assistant.\n\n"\
           "Your task is read the following text and determine which one "\
           "of these categories best describes its author:\n\n"\
           "Religious Conspiracy Theorist\n"\
           "Misinformation Spreader\n"\
           "Anti Vaxxer\n"\
           "Fear Mongerer\n\n"\
           "Text:\n" + text + "\n"\
           "Respond with exactly one of these labels—and nothing else: "

@torch.no_grad()
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(phi3_model.device)

    output = phi3_model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False  # Fix for DynamicCache issue
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    reply = decoded.strip()
    responses.append(reply)

    reply_lower = reply.lower()
    if "conspiracy" in reply_lower:
        return 0
    elif "misinformation" in reply_lower:
        return 1
    elif "vaxx" in reply_lower or "vaccine" in reply_lower:
        return 2
    elif "fear" in reply_lower:
        return 3
    else:
        return -1

def main():
    base_folder = '/kaggle/input/owaiskkk/VaxGaurd-Dataset-roles (New)/VaxGaurd-Dataset-roles (New)/'
    files = [
        [
            'GPT-3.5/INFLUENZA/clean_Influenza_texts_GPT-3.5_Anti-Vacciner.csv',
            'GPT-3.5/INFLUENZA/clean_Influenza_texts_GPT-3.5_Misinformation spreader.csv',
            'GPT-3.5/INFLUENZA/clean_Influenza_texts_GPT-3.5_fear mongerer.csv',
            'GPT-3.5/INFLUENZA/clean_Influenza_texts_GPT-3.5_religious Conspiracy theorist.csv'
        ],
        [
            'GPT-4o/Influenza/clean_Influenza_texts_GPT-4o_Anti-Vacciner.csv',
            'GPT-4o/Influenza/clean_Influenza_texts_GPT-4o_Misinformation spreader.csv',
            'GPT-4o/Influenza/clean_Influenza_texts_GPT-4o_fear mongerer.csv',
            'GPT-4o/Influenza/clean_Influenza_texts_GPT-4o_religious Conspiracy theorist.csv'
        ],
        [
            'Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Anti-Vacciner.csv',
            'Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Misinformation spreader.csv',
            'Llama 3/INFLUENZA/clean_Influenza_texts_llama_fear_monger.csv',
            'Llama 3/INFLUENZA/clean_Influenza_texts_llama_reilgious.csv'
        ],
        [
            'Mistral/INFLUENZA/clean_Influenza_texts_mistral_Anti-Vacciner.csv',
            'Mistral/INFLUENZA/clean_Influenza_texts_mistral_Misinformation spreader.csv',
            'Mistral/INFLUENZA/clean_Influenza_texts_mistral_fear.csv',
            'Mistral/INFLUENZA/clean_Influenza_texts_mistral_religious.csv'
        ]
    ]

    # ✅ Corrected indentation
    files = [[base_folder + csv_file for csv_file in model] for model in files]

    models = ['GPT-3.5', 'GPT-4o', 'Llama 3', 'Mistral']
    for i, model_files in enumerate(files):
        texts = []
        labels = []
        model = models[i]

        for j, file in enumerate(model_files):
            df = pd.read_csv(file, encoding='latin1')
            col = 'Misinformation' if 'Misinformation' in df.columns else df.columns[0]
            df = df[col].dropna()
            texts.extend(df.tolist())
            labels.extend([j] * len(df))

        preds = []
        for text in tqdm(texts, desc=f'Model: {model}'):
            preds.append(classify(text))

        with open(f'responses_COVID_{model}.pickle', 'wb') as f:
            pickle.dump(responses, f)
            responses.clear()

        with open(f'COVID_{model}.pickle', 'wb') as f:
            pickle.dump(texts, f)
            pickle.dump(labels, f)
            pickle.dump(preds, f)

        overall_acc = accuracy_score(labels, preds)
        overall_prec = precision_score(labels, preds, average="macro", zero_division=0)
        overall_rec = recall_score(labels, preds, average="macro", zero_division=0)
        overall_f1 = f1_score(labels, preds, average="macro", zero_division=0)

        print(f'\n{model} Results:')
        print(f'Accuracy:  {overall_acc:.4f}')
        print(f'Precision: {overall_prec:.4f}')
        print(f'Recall:    {overall_rec:.4f}')
        print(f'F1 Score:  {overall_f1:.4f}')

if __name__ == '__main__':
    main()
